In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg

from PIL import Image

import os
from pathlib import Path

import re

from collections import OrderedDict

import base64
from io import BytesIO
from skimage import io

from tqdm.auto import tqdm

from math import pi

from bokeh.models import Legend, LegendItem, LabelSet, ColumnDataSource
from bokeh.io import output_file, output_notebook, curdoc
from bokeh.plotting import figure, show
from bokeh.transform import cumsum
output_notebook()

# Simple EDA

In [3]:
pokemon = pd.read_csv("/kaggle/input/pokemon/Pokemon.csv")

In [4]:
pokemon.head()

In [5]:
pokemon.columns

In [6]:
pokemon.columns = ['id', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense', 
                   'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary']
pokemon.columns

In [7]:
pokemon.tail()

# Data Visualization

In [8]:
pokemon['Dual-Type'] = pokemon['Type 1'].fillna('') + ' ' + pokemon['Type 2'].fillna('')
pokemon['Dual-Type'] = pokemon['Dual-Type'].str.strip()

print(f"Number of types in Type 1: {len(pokemon['Type 1'].dropna().unique())}")
print(f"Number of types in Type 2: {len(pokemon['Type 2'].dropna().unique())}")
print(f"Number of types in Dual-Type: {len(pokemon['Dual-Type'].dropna().unique())}")

In [9]:
pokemon['Dual-Type']

In [10]:
count_type_1 = pokemon.groupby(by = ['Generation', 'Type 1'], dropna = True).count()[['id']]

count_type_1.reset_index(inplace = True)

count_type_1.rename(columns = {'Type 1' : 'Type'}, inplace = True)

count_type_1.set_index(['Generation', 'Type'], inplace = True)

count_type_1

In [11]:
count_type_2 = pokemon.groupby(by = ['Generation', 'Type 2'], dropna = True).count()[['id']]

count_type_2.reset_index(inplace = True)

count_type_2.rename(columns = {'Type 2' : 'Type'}, inplace = True)

count_type_2.set_index(['Generation', 'Type'], inplace = True)

count_type_2

In [12]:
count_single_type = count_type_1.add(count_type_2, axis = 'columns', fill_value = 0)

del count_type_1, count_type_2

count_single_type['id'] = count_single_type['id'].astype('int')

count_single_type.reset_index(level = [0, 1], inplace = True)

count_single_type

In [13]:
#count_single_type.loc['all'] = df.apply(lambda x : x.groupby(by = ['Type'])['id'].sum())

sum_single_type = pd.pivot_table(count_single_type, index = ['Type'], values = ['id'], aggfunc=np.sum)

sum_single_type['id'] = sum_single_type['id'].astype('int')

sum_single_type.reset_index(inplace = True)

sum_single_type['Generation'] = 'all'

single_type = pd.concat([count_single_type, sum_single_type])  

del sum_single_type

single_type.reset_index(drop = True, inplace = True)

single_type

## matplotlib

In [14]:
fig, ax = plt.subplots(figsize = (10, 10))

def creat_pokeball(size, generation):
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    Pokeball_Crimson = '#EE1515'
    Pokeball_Shark = '#222224'
    Pokebll_Gallery = '#F0F0F0'
    
    palette_generation_1 = ['#FF7327', '#00DD00']
    palette_generation_2 = ['#B69E00', '#C0C0E1']
    palette_generation_3 = ['#AB2813', '#26649C']
    palette_generation_4 = ['#44BAE5', '#DA7D99']
    palette_generation_5 = ['#444444', '#E1E1E1']
    palette_generation_6 = ['#025DA6', '#EA1A3E']
    palette_generation_all = [Pokeball_Crimson, Pokebll_Gallery]
    palettes = [palette_generation_1, palette_generation_2, palette_generation_3,
                palette_generation_4, palette_generation_5, palette_generation_6,
                palette_generation_all]
    if generation in [1, 2, 3, 4, 5, 6]:
        pokeball_colors = palettes[generation - 1]
    elif generation == 'all':
        pokeball_colors = palettes[6]
    else:
        raise ValueError("The entered generation information is incorrect.")
        
    wedges_0, texts_0 = ax.pie(
        x = [100],
        labels = ['GEN\n' + str(generation).upper()],
        colors = [Pokebll_Gallery],
        startangle = 0,
        explode = [0] * len([100]),
        wedgeprops = {'width' : 1.15 * size},
        radius = 1.15 * size,
    )
    
    for i, wedge in enumerate(wedges_0):
        texts_0[i].set_color(Pokeball_Crimson)
        texts_0[i].set_fontsize(24 * size)
        texts_0[i].set_fontstyle('normal')
        texts_0[i].set_fontweight('roman')
        texts_0[i].set_horizontalalignment('center')
        texts_0[i].set_position((0, 0))
        texts_0[i].set_alpha(0.75)
        
    wedges_1 = ax.pie(
        x = [100],
        colors = [Pokeball_Shark],
        startangle = 0,
        explode = [0] * len([100]),
        wedgeprops = {'width': 0.8 * size},
        radius = 1.15 * size,
    )
    
    wedges_2 = ax.pie(
        x = [50, 50],
        colors = pokeball_colors,
        startangle = 0,
        explode = [0.05 * size] * len([50, 50]),
        wedgeprops = {'width': 0.85 * size, 'linewidth': 8 * size, 'edgecolor': Pokeball_Shark},
        radius = 1.2 * size,
    )
    
    wedges_3 = ax.pie(
        x = [100],
        colors = [Pokebll_Gallery],
        explode = [0] * len([100]),
        startangle = 0,
        wedgeprops = dict(width = 0.05 * size, linewidth = 16 * size, edgecolor = Pokeball_Shark),
        radius = 0.4 * size,
    )
    
    wedges_4 = ax.pie(
        x = [100],
        colors = [Pokebll_Gallery],
        explode = [0] * len([100]),
        startangle = 0,
        wedgeprops = dict(width = 0.25 * size, linewidth = 4 * size, edgecolor = Pokeball_Shark),
        radius = 0.25 * size,
    )

creat_pokeball(1, 'all')

plt.show()

In [15]:
def starter_classifier():
    n_generation_1 = ['001', '004', '007']
    n_generation_2 = ['152', '155', '158']
    n_generation_3 = ['252', '255', '258']
    n_generation_4 = ['387', '390', '393']
    n_generation_5 = ['495', '498', '501']
    n_generation_6 = ['650', '653', '656']
    n_generation_All = ['025', '133']
    
    image_generation_1 = {}
    image_generation_2 = {}
    image_generation_3 = {}
    image_generation_4 = {}
    image_generation_5 = {}
    image_generation_6 = {}
    image_generation_All = {}
    
    for dirname, _, filenames in os.walk('../input/pokemon-starters-by-generation/'):
        for filename in filenames:
            n_pokedex = re.split('\.|_', filename)[1]
            if n_pokedex in n_generation_1:
                image_generation_1.update({n_pokedex: os.path.join(dirname, filename)})
            elif n_pokedex in n_generation_2:
                image_generation_2.update({n_pokedex: os.path.join(dirname, filename)})
            elif n_pokedex in n_generation_3:
                image_generation_3.update({n_pokedex: os.path.join(dirname, filename)})
            elif n_pokedex in n_generation_4:
                image_generation_4.update({n_pokedex: os.path.join(dirname, filename)})
            elif n_pokedex in n_generation_5:
                image_generation_5.update({n_pokedex: os.path.join(dirname, filename)})
            elif n_pokedex in n_generation_6:
                image_generation_6.update({n_pokedex: os.path.join(dirname, filename)})
            else:
                image_generation_All.update({n_pokedex: os.path.join(dirname, filename)})
                
    return image_generation_1, image_generation_2, image_generation_3, image_generation_4, image_generation_5, image_generation_6, image_generation_All

In [16]:
fig, ax = plt.subplots(figsize = (10, 10))

def add_starters(size, generation):
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    creat_pokeball(size, generation)
    ns_pokedex = []
    image_paths = []
    if generation in [1, 2, 3, 4, 5, 6]:
        dict_starters = starter_classifier()[generation - 1]
        for key in sorted(dict_starters):
            ns_pokedex.append(key)
            image_paths.append(dict_starters[key])
        image_1 = mpimg.imread(image_paths[0])
        image_2 = mpimg.imread(image_paths[1])
        image_3 = mpimg.imread(image_paths[2])
        imagebox_1 = OffsetImage(image_1, zoom = .45 * size)
        imagebox_2 = OffsetImage(image_2, zoom = .45 * size)
        imagebox_3 = OffsetImage(image_3, zoom = .45 * size)
        annotationbox_1 = AnnotationBbox(imagebox_1, (-.52 * size, -.3 * size), frameon = False)
        annotationbox_2 = AnnotationBbox(imagebox_2, (0 * size, .6 * size), frameon = False)
        annotationbox_3 = AnnotationBbox(imagebox_3, (.52 * size, -.3 * size), frameon = False)
        ax.add_artist(annotationbox_1)
        ax.add_artist(annotationbox_2)
        ax.add_artist(annotationbox_3)
    elif generation == 'all':
        dict_starters = starter_classifier()[6]
        for key in sorted(dict_starters):
            ns_pokedex.append(key)
            image_paths.append(dict_starters[key])
        image_1 = mpimg.imread(image_paths[0])
        image_2 = mpimg.imread(image_paths[1])
        imagebox_1 = OffsetImage(image_1, zoom = .4 * size)
        imagebox_2 = OffsetImage(image_2, zoom = .4 * size)
        annotationbox_1 = AnnotationBbox(imagebox_1, (-.52 * size, -.3 * size), frameon = False)
        annotationbox_2 = AnnotationBbox(imagebox_2, (.52 * size, -.3 * size), frameon = False)
        ax.add_artist(annotationbox_1)
        ax.add_artist(annotationbox_2)
    else:
        raise ValueError("The entered generation information is incorrect.")

add_starters(1, 'all')

plt.show()

In [17]:
fig, ax = plt.subplots(figsize = (10, 10))

def save_to_image(size, generation):
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    parent_dir = '/kaggle/working'
    directory = 'pokeballs'
    joint_path = os.path.join(parent_dir, directory)
    abs_path = os.path.abspath(joint_path)
    Path(abs_path).mkdir(parents = True, exist_ok = True)
    
    add_starters(size, generation)
    save_path = abs_path + '/pokeball_' + str(generation) + '.png'
    plt.savefig(fname = save_path, transparent = True)
    plt.cla()

for generation in [1, 2, 3, 4, 5, 6, 'all']:
    save_to_image(1, generation)

plt.clf()
plt.close()

In [18]:
fig, ax = plt.subplots(figsize = (10, 10))

def pie_chart(size, generation, df, pctdistance, font_size_weight):
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    Pokebll_Gallery = '#F0F0F0'
    
    df_generation = df.loc[df['Generation'] == generation].copy()
    sorted_types = df_generation.sort_values(by = 'id')['Type']
    
    proportion_all = zip(single_type.loc[single_type['Generation'] == 'all'].sort_values(by = 'id')['id'] /
                         single_type.loc[single_type['Generation'] == 'all'].sort_values(by = 'id')['id'].sum(),
                     single_type.loc[single_type['Generation'] == 'all'].sort_values(by = 'id')['Type'])
    sorted_types_all = {}
    for i , j in proportion_all:
        sorted_types_all.update({j : i})
    sorted_types_all = OrderedDict([(i, sorted_types_all[i]) for i in sorted_types])
    
    colors = {
        'Bug': '#A8B820',
        'Dark': '#705848',
        'Dragon': '#7038F8',
        'Electric': '#F8D030',
        'Fairy': '#F0B6BC',
        'Fighting': '#C03028',
        'Fire': '#F08030',
        'Flying': '#A890F0',
        'Ghost': '#705898',
        'Grass': '#78C850',
        'Ground': '#E0C068',
        'Ice': '#98D8D8',
        'Normal': '#A8A878',
        'Poison': '#A040A0',
        'Psychic': '#F85888',
        'Rock': '#B8A038',
        'Steel': '#B8B8D0',
        'Water': '#6890F0'
    }
    
    df_generation['colors'] = df_generation['Type'].apply(lambda x: colors[x])
    
    edgecolors = {
        'Bug': '#D8E030',
        'Dark': '#A8A878',
        'Dragon': '#B8A0F8',
        'Electric': '#F8F878',
        'Fairy': '#F5CAD1',
        'Fighting': '#F08030',
        'Fire': '#F8D030',
        'Flying': '#C8C0F8',
        'Ghost': '#A890F0',
        'Grass': '#C0F860',
        'Ground': '#F8F878',
        'Ice': '#D0F8E8',
        'Normal': '#D8D8D0',
        'Poison': '#D880B8',
        'Psychic': '#F8C0B0',
        'Rock': '#E0C068',
        'Steel': '#D8D8C0',
        'Water': '#98D8D8'
    }
    
    wedges_1, texts_1, autotexts_1 = ax.pie(
        x = df_generation.sort_values(by = 'id')['id'],
        labels = df_generation.sort_values(by = 'id')['Type'],
        colors = df_generation.sort_values(by = 'id')['colors'],
        autopct = '%.2f%%',
        startangle = 0,
        labeldistance = 0,
        pctdistance = pctdistance,
        wedgeprops = dict(linewidth = 2 * size, edgecolor = Pokebll_Gallery, alpha = 0.75),
        radius = 2.5 * size,
    )
    
    for i, wedge in enumerate(wedges_1):
        wedges_1[i].set_edgecolor(edgecolors.get(wedge.get_label()))
    
    for i in texts_1:
        i.set(visible = False)
    
    for wedge, i in zip(wedges_1, sorted_types_all):
        wedge.set_radius((1.2 + 10 * sorted_types_all.get(i)) * size)
    
    count = 0
    for autotext, i in zip(autotexts_1, sorted_types_all):
        autotext.set_fontsize((sorted_types_all.get(i) + count) * size * font_size_weight)
        count += 0.0001
    
    [t.set_color(Pokebll_Gallery) for t in autotexts_1]
    
    abs_path = '/kaggle/working/pokeballs'
    img_path = abs_path + '/pokeball_' + str(generation) + '.png'

    fig = mpimg.imread(img_path, format = 'png')
    imagebox = OffsetImage(fig, zoom = .8 * size)
    annotationbox = AnnotationBbox(imagebox, (0, 0), frameon = False)
    ax.add_artist(annotationbox)

pie_chart(1, 'all', single_type, .56, 300)

plt.show()

In [19]:
fig = plt.figure(figsize = (30, 30))

ax = fig.add_gridspec(3, 3)
axes = []
axes.append(fig.add_subplot(ax[0, 0]))
axes.append(fig.add_subplot(ax[0, 1]))
axes.append(fig.add_subplot(ax[0, 2]))
axes.append(fig.add_subplot(ax[2, 0]))
axes.append(fig.add_subplot(ax[2, 1]))
axes.append(fig.add_subplot(ax[2, 2]))
axes.append(fig.add_subplot(ax[1, 0: ]))

Pokeball_Crimson = '#EE1515'
Pokeball_Shark = '#222224'
Pokebll_Gallery = '#F0F0F0'
    
colors = {
        'Bug': '#A8B820',
        'Dark': '#705848',
        'Dragon': '#7038F8',
        'Electric': '#F8D030',
        'Fairy': '#F0B6BC',
        'Fighting': '#C03028',
        'Fire': '#F08030',
        'Flying': '#A890F0',
        'Ghost': '#705898',
        'Grass': '#78C850',
        'Ground': '#E0C068',
        'Ice': '#98D8D8',
        'Normal': '#A8A878',
        'Poison': '#A040A0',
        'Psychic': '#F85888',
        'Rock': '#B8A038',
        'Steel': '#B8B8D0',
        'Water': '#6890F0'
    }

for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

for i in [1, 2, 3, 4, 5, 6, 'all']:
    if i == 'all':
        ax = axes[6]
        sub = pie_chart(.85, i, single_type, .575, 275)
        df_generation = single_type.loc[single_type['Generation'] == i]
        legend = ax.legend(title = 'Pokemon Types',
                    title_fontsize = 24,
                    bbox_to_anchor = (0, 0, 2.2, 1.12),
                    labels = "Type " + df_generation.sort_values(by = 'id')['Type'].astype('str'),
                    labelcolor = df_generation.sort_values(by = 'id')['Type'].apply(lambda x: colors[x]),
                    fontsize = 20,
                    borderpad = 1,
                    facecolor = Pokebll_Gallery,
                    edgecolor = Pokeball_Shark)
        plt.setp(legend.get_title(),
                 color = Pokeball_Crimson,
                 fontstyle = 'normal',
                 fontfamily = 'monospace')
    else:
        ax = axes[i - 1]
        sub = pie_chart(.7, i, single_type, .585, 250)

title = fig.suptitle("Distribution of Pokemon Types in Different Generations",
                     fontweight = 'black',
                     fontstretch = 'expanded',
                     fontstyle = 'normal',
                     fontfamily = 'monospace',
                     fontsize = 30,
                     y = .9,
                     color = Pokeball_Crimson)

fig.savefig('pokemon_type_analysis_matplotlib.png', transparent = True)

plt.show()

## bokeh

In [20]:
def to_png(arr):
    out = BytesIO()
    im = Image.fromarray(arr)
    im.save(out, format = 'png')
    return out.getvalue()

def b64_image_files(images):
    urls = []
    for im in tqdm(images):
        png = to_png(im)
        url = 'data:image/png;base64,' + base64.b64encode(png).decode('utf-8')
        urls.append(url)
    return urls

image_url = []
for i in ['1', '2', '3', '4', '5', '6', 'all']:
    images = [io.imread('./pokeballs/pokeball_' + i + '.png')]
    image_url.append(b64_image_files(np.array(images)))

In [21]:
output_file("pokemon_type_analysis_of_one_generation_bokeh.html", title = "Pokemon Type Analysis of One Generation")

TOOLTIPS = [
    ('TYPE', '@Type'),
    ('NUM', '@id'),
    ('PERCENTAGE', '@percentage')
]

plot = figure(width = 900, height = 900,
              title = "Pokemon Type Analysis of One Generation",
              tools = 'pan, hover, wheel_zoom, box_zoom, reset, save',
              tooltips = TOOLTIPS,
              x_range = (-1.0, 1.0),
              y_range = (-1.0, 1.0))

def single_generation_bokeh(generation, x = 0, y = 0, legend = True):
    colors = {
        'Bug': '#A8B820',
        'Dark': '#705848',
        'Dragon': '#7038F8',
        'Electric': '#F8D030',
        'Fairy': '#F0B6BC',
        'Fighting': '#C03028',
        'Fire': '#F08030',
        'Flying': '#A890F0',
        'Ghost': '#705898',
        'Grass': '#78C850',
        'Ground': '#E0C068',
        'Ice': '#98D8D8',
        'Normal': '#A8A878',
        'Poison': '#A040A0',
        'Psychic': '#F85888',
        'Rock': '#B8A038',
        'Steel': '#B8B8D0',
        'Water': '#6890F0'
    }
    
    edgecolors = {
        'Bug': '#D8E030',
        'Dark': '#A8A878',
        'Dragon': '#B8A0F8',
        'Electric': '#F8F878',
        'Fairy': '#F5CAD1',
        'Fighting': '#F08030',
        'Fire': '#F8D030',
        'Flying': '#C8C0F8',
        'Ghost': '#A890F0',
        'Grass': '#C0F860',
        'Ground': '#F8F878',
        'Ice': '#D0F8E8',
        'Normal': '#D8D8D0',
        'Poison': '#D880B8',
        'Psychic': '#F8C0B0',
        'Rock': '#E0C068',
        'Steel': '#D8D8C0',
        'Water': '#98D8D8'
    }
    
    df_generation = single_type.loc[single_type['Generation'] == generation].copy()
    
    sorted_types = df_generation.sort_values(by = 'id')['Type']
    proportion_all = zip(single_type.loc[single_type['Generation'] == 'all'].sort_values(by = 'id')['id'] /
                             single_type.loc[single_type['Generation'] == 'all'].sort_values(by = 'id')['id'].sum(),
                         single_type.loc[single_type['Generation'] == 'all'].sort_values(by = 'id')['Type'])
    sorted_types_all = {}
    for i , j in proportion_all:
        sorted_types_all.update({j : i})
    sorted_types_all = OrderedDict([(i, sorted_types_all[i]) for i in sorted_types])
    
    df_generation = df_generation.sort_values(by = 'id').reset_index()
    df_generation['angle'] = df_generation['id'] / df_generation['id'].sum() * 2 * pi
    df_generation['color'] = df_generation['Type'].apply(lambda x: colors[x])
    df_generation['edgecolors'] = df_generation['Type'].apply(lambda x: edgecolors[x])
    df_generation['percentage'] = df_generation['id'] / df_generation['id'].sum() * 100
    
    float_format = lambda df: '{:,.2f}%'.format(df['percentage'])
    df_generation['percentage'] = df_generation.apply(float_format, axis = 1)

    start_angle = pi / 2
    end_angle = pi / 2
    legend_items = []
    labels_list = []
    for i in sorted_types_all:
        start_angle = start_angle
        end_angle += df_generation.loc[df_generation['Type'] == i]['angle'].values[0]
        renderer = plot.annular_wedge(x = x, y = y, inner_radius = 100,
                                      outer_radius = (1 + sorted_types_all.get(i) * 20) * 100,
                                      outer_radius_units = 'screen', inner_radius_units = 'screen',
                                      start_angle = start_angle, end_angle = end_angle,
                                      line_color = 'edgecolors', fill_color = 'color',
                                      source = df_generation.loc[df_generation['Type'] == i], alpha = .6)
        legend_items.append(LegendItem(label = 'Type ' + i, renderers = [renderer]))
        
        df_generation['percentage'] = df_generation['percentage'].astype(str)
        df_generation['percentage'] = df_generation['percentage'].str.pad(38, side = 'left')
        source = ColumnDataSource(df_generation.loc[df_generation['Type'] == i])
        label = LabelSet(x = x, y = y, text = 'percentage',
                         angle = start_angle,
                         text_font_size = str(10 + round(sorted_types_all.get(i) * 100)) + 'px',
                         text_color = 'edgecolors',
                         source = source, render_mode = 'canvas')
        plot.add_layout(label)
        
        start_angle = end_angle
    
    if legend == False:
        pass
    else:
        legend = Legend(items = legend_items, click_policy = 'mute',
                        location = 'top_left', label_text_font_size = '12px')
        plot.add_layout(legend, 'right')

    if generation == 'all':
        plot.image_url(
            url=[image_url[-1]],
            x = x, y = y,
            h = 300, h_units = 'screen',
            w = 300, w_units = 'screen',
            anchor = 'center'
        )
    else:
        plot.image_url(
            url=[image_url[generation - 1]],
            x = x, y = y,
            h = 300, h_units = 'screen',
            w = 300, w_units = 'screen',
            anchor = 'center'
        )
    
    doc = curdoc()
    doc.add_root(plot)

single_generation_bokeh('all')

plot.axis.axis_label = None
plot.axis.visible = False
plot.grid.grid_line_color = None

show(plot)

In [22]:
output_file("pokemon_type_analysis_of_all_generations_bokeh.html",
            title = "Pokemon Type Analysis of All Generations")

TOOLTIPS = [
    ('TYPE', '@Type'),
    ('NUM', '@id'),
    ('PERCENTAGE', '@percentage')
]

plot = figure(width = 2000, height = 1600,
              title = "Pokemon Type Analysis of All Generations",
              tools = 'pan, hover, wheel_zoom, box_zoom, reset, save',
              tooltips = TOOLTIPS,
              x_range = (-5.0, 5.0),
              y_range = (-4.0, 4.0))

coordinate = [(.866, -.5), (.866, .5), (0, 1), (-.866, .5), (-.866, -.5), (0, -1)]
for generation in [1, 2, 3, 4, 5, 6, 'all']:
    if generation == 'all':
        single_generation_bokeh(generation)
    else:
        single_generation_bokeh(generation,
                                x = coordinate[generation - 1][1] * 3,
                                y = coordinate[generation - 1][0] * 3,
                                legend = False)

plot.axis.axis_label = None
plot.axis.visible = False
plot.grid.grid_line_color = None

show(plot)

## plotly